<a href="https://colab.research.google.com/github/Rijan-Joshi/Fun-Learning/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
#Importing the data

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    transform = ToTensor(),
    download= True
)

test_data = datasets.FashionMNIST(
    root='data',
    train = False,
    transform = ToTensor(),
    download = True
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 274kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 4.50MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 30.0MB/s]


In [ ]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

X, y = next(iter(test_dataloader))
X.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [ ]:
#Creating Model in PyTorch

device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
print(f"Using {device} device")

#Defining the model

class NeuralNetwork(nn.Module):

  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x) #Raw Output
    return logits


Using cpu device


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
#Loss Functions and Optimizers
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [ ]:
#Start training

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


    if batch % 100 == 0:
      loss, current = loss.item(), (batch +1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
#Define testing function

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)

      #Prediction
      y_pred = model(X)
      test_loss += loss_fn(y_pred, y).item()
      correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  accuracy = correct / size

  print(f'Test Error: \n Accuracy {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n')



In [ ]:
#Start Training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.304952  [   64/60000]
loss: 2.297890  [ 6464/60000]
loss: 2.280012  [12864/60000]
loss: 2.273349  [19264/60000]
loss: 2.243645  [25664/60000]
loss: 2.221744  [32064/60000]
loss: 2.225010  [38464/60000]
loss: 2.190857  [44864/60000]
loss: 2.198745  [51264/60000]
loss: 2.158017  [57664/60000]
Test Error: 
 Accuracy 42.9%, Avg loss: 2.157882 

Epoch 2
-------------------------------
loss: 2.168061  [   64/60000]
loss: 2.160018  [ 6464/60000]
loss: 2.111921  [12864/60000]
loss: 2.123873  [19264/60000]
loss: 2.064968  [25664/60000]
loss: 2.012007  [32064/60000]
loss: 2.035507  [38464/60000]
loss: 1.965997  [44864/60000]
loss: 1.975939  [51264/60000]
loss: 1.892624  [57664/60000]
Test Error: 
 Accuracy 56.2%, Avg loss: 1.901467 

Epoch 3
-------------------------------
loss: 1.936215  [   64/60000]
loss: 1.901791  [ 6464/60000]
loss: 1.801145  [12864/60000]
loss: 1.832533  [19264/60000]
loss: 1.709518  [25664/60000]
loss: 1.675409  [32064/60000

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[7][0], test_data[7][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  pred_idx = pred[0].argmax()
  print(f"Predicted: {classes[pred_idx]}, Actual: {classes[y]}")



Predicted: Coat, Actual: Shirt


In [5]:
torch.ones(4)

tensor([1., 1., 1., 1.])

In [9]:
r = torch.rand(2,3) - 0.5 * 2
r


tensor([[-0.4282, -0.9554, -0.0652],
        [-0.7766, -0.1761, -0.5614]])